In [6]:
import pandas as pd

# emails.csv from 

import csv
file_path_email = 'data/emails.csv'

#Are there null weeks?
emails = pd.read_csv(file_path_email) #https://github.com/PracticalTimeSeriesAnalysis/BookRepo/blob/master/Ch02/data/emails.csv
emails['week'] = pd.to_datetime(emails['week'])
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [7]:
#Are null weeks actually reported?
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


In [8]:
# We can see that some weeks are missing. How many weeks are there between the first and last event for member 998?
# First, we calculate the member's membership duration in weeks
(max(emails[emails.user == 998].week) - min(emails[emails.user == 998].week)).days/7

25.0

In [9]:
#How many corresponding weeks of data do we have for this member:
emails[emails.user == 998].shape

(24, 3)

In [10]:
# There should be 26 lines, so there is a missing week. Fill in the missing week:
#1) Create a Cartesian product with the combination of users and weeks
complete_idx = pd.MultiIndex.from_product((set(emails.week),set(emails.user)))
complete_idx

MultiIndex([('2015-06-01',   1.0),
            ('2015-06-01',   3.0),
            ('2015-06-01',   5.0),
            ('2015-06-01',   6.0),
            ('2015-06-01',   9.0),
            ('2015-06-01',  10.0),
            ('2015-06-01',  14.0),
            ('2015-06-01',  16.0),
            ('2015-06-01',  20.0),
            ('2015-06-01',  21.0),
            ...
            ('2015-08-31', 973.0),
            ('2015-08-31', 977.0),
            ('2015-08-31', 982.0),
            ('2015-08-31', 984.0),
            ('2015-08-31', 987.0),
            ('2015-08-31', 991.0),
            ('2015-08-31', 992.0),
            ('2015-08-31', 993.0),
            ('2015-08-31', 995.0),
            ('2015-08-31', 998.0)],
           length=93247)

In [11]:
# 2) Use the Cartesian product to reindex the original table and fill in the missing values:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()

all_email.columns = ['week','user','emailsOpened']

In [12]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
74920,2015-02-09,998.0,0.0
84622,2015-02-16,998.0,0.0
18864,2015-02-23,998.0,0.0
28027,2015-03-02,998.0,0.0
75459,2015-03-09,998.0,0.0
...,...,...,...
70608,2018-04-30,998.0,3.0
50665,2018-05-07,998.0,3.0
41502,2018-05-14,998.0,3.0
23715,2018-05-21,998.0,3.0


In [15]:
# User membership start cutoff point:
# 1) Find start_date and end_date
cutoff_dates = emails.groupby('user').week.agg(['min','max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

In [21]:
#Drop the DataFrame rows that don't contribute much to the timeline
for _, row in cutoff_dates.iterrows():
    member = row['user']
    start_date = row['min']
    end_date = row['max']
    
    drop_indices = all_email[
        (all_email['user'] == member) & (all_email['week'] < start_date)
    ].index
    
    drop_indices = all_email[
        (all_email['user'] == member) & (all_email['week'] > end_date)
    ].index
    
    all_email.drop(drop_indices, inplace=True)

In [34]:
all_email

,week,user,emailsOpened
5,2015-06-01,10.0,0.0
13,2015-06-01,31.0,0.0
16,2015-06-01,38.0,0.0
25,2015-06-01,54.0,0.0
34,2015-06-01,75.0,0.0
...,...,...,...
93221,2015-08-31,940.0,1.0
93231,2015-08-31,959.0,0.0
93238,2015-08-31,977.0,0.0
93239,2015-08-31,982.0,0.0


In [26]:
# Relate email and donation data to each other - do p dpwmsampling of the donation data to transform it into a weekly time series that is comparable

import csv
file_path_donations = 'data/donations.csv'

#Are there null weeks?
donations = pd.read_csv(file_path_donations) #https://github.com/PracticalTimeSeriesAnalysis/BookRepo/blob/master/Ch02/data/emails.csv
donations.timestamp = pd.to_datetime(donations.timestamp)
#Set timestamp to index
donations.set_index('timestamp', inplace = True)

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0
...,...,...
2016-09-02 11:20:00,25.0,992.0
2017-11-02 12:17:06,50.0,993.0
2016-09-13 21:09:47,1000.0,995.0


In [52]:
agg_don = (
    donations
    .groupby('user')
    .apply(lambda df: df.resample("W-MON")['amount'].sum().dropna(), include_groups=False)
    .reset_index()
)

In [63]:
# Emails and user-aligned action data

merged_df = pd.DataFrame()  # Initialize an empty DataFrame

for user, user_email in all_email.groupby('user'):
    user_donations = agg_don[agg_don['user'] == user].copy()  # Filter donations for the user
    user_donations.set_index('timestamp', inplace=True)  # Reset the index to timestamp
    
    user_email = all_email[all_email['user'] == user].copy()  # Filter emails for the user
    user_email.set_index('week', inplace=True)  # Reset the index to week
    
    # Perform the merge of the DataFrames
    df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
    df = df.fillna(0)  # Fill null values with 0
    df['user'] = df['user_x']  # Add the 'user' column
    
    # Add the results to merged_df using pd.concat
    merged_df = pd.concat(
        [merged_df, df.reset_index()[['user', 'week', 'emailsOpened', 'amount']]],
        ignore_index=True
    )


In [64]:
# Shift donations one week ahead to align email reading behavior with donations
df = merged_df[merged_df.user == 998]
df.loc[:, 'target'] = df.amount.shift(1)
df = df.fillna(0)
df

C:\Users\gdavid\AppData\Local\Temp\ipykernel_9108\856631622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.amount.shift(1)


,user,week,emailsOpened,amount,target
42313,998.0,2018-01-08,3.0,50.0,0.0
42314,998.0,2018-04-09,3.0,0.0,50.0
42315,998.0,2017-12-18,3.0,0.0,0.0
42316,998.0,2018-01-22,3.0,0.0,0.0
42317,998.0,2018-05-21,3.0,0.0,0.0
42318,998.0,2017-12-25,0.0,0.0,0.0
42319,998.0,2018-01-15,2.0,0.0,0.0
42320,998.0,2018-05-28,3.0,0.0,0.0
42321,998.0,2018-05-14,3.0,0.0,0.0
42322,998.0,2018-03-19,2.0,0.0,0.0
